In [724]:
colors = {
    'white': 0,
    'black': 1,
}
figures = {
    'Pawn': 1,
    'Rook': 2,
    'Knight': 3,
    'Bishop': 4,
    'Queen': 5,
    'King': 6,
}
reverse_figures = {v: k[0] for k, v in figures.items()}

In [725]:
class Piece:
    def __init__(self, color, position):
        self.color = color
        self.position = position
        self.eliminated = False
        self.destructable = True

    def eliminate(self):
        self.eliminated = True

In [726]:
class Pawn(Piece):
    def __init__(self, color, position):
        super().__init__(color, position)
        self.first_move = True
        self.figure = figures['Pawn']

    def valid_moves(self, board_instance):
        moves = []
        row, col = self.position

        if self.color == colors['white']:
            if row == 1 and not board_instance.check_for_friendly((row + 2, col), self.color):
                moves.append((row + 2, col))
            if row + 1 < 8 and not board_instance.check_for_friendly((row + 1, col), self.color):
                moves.append((row + 1, col))
            dc = [[1, 1], [1, -1]]
            for i in range(2):
                new_row, new_col = row + dc[i][0], col + dc[i][1]
                if 0 <= new_row < 8 and 0 <= new_col < 8 and not board_instance.check_for_friendly((new_row, new_col), self.color):
                    moves.append((new_row, new_col))

        if self.color == colors['black']:
            if row == 6 and not board_instance.check_for_friendly((row - 2, col), self.color):
                moves.append((row - 2, col))
            if row - 1 >= 0 and not board_instance.check_for_friendly((row - 1, col), self.color):
                moves.append((row - 1, col))
            dc = [[-1, 1], [-1, -1]]
            for i in range(2):
                new_row, new_col = row + dc[i][0], col + dc[i][1]
                if 0 <= new_row < 8 and 0 <= new_col < 8 and not board_instance.check_for_friendly((new_row, new_col), self.color):
                    moves.append((new_row, new_col))

        return moves

In [727]:
class Rook(Piece):
    def __init__(self, color, position):
        super().__init__(color, position)
        self.first_move = True
        self.figure = figures['Rook']

    def valid_moves(self, board_instance):
        moves = []
        row, col = self.position

        directions = [[1, 0], [-1, 0], [0, 1], [0, -1]]

        for dr,dc in directions:
            r,c = row +dr , col + dc
            while 0 <= r < 8 and 0 <= c < 8:

                if board_instance.check_for_friendly((r,c), self.color):
                    break
                moves.append((r,c))
                if board_instance.check_for_enemy((r,c), self.color):
                    break
                r += dr
                c += dc
        return moves


In [728]:
class Knight(Piece):
    def __init__(self, color, position):
        super().__init__(color, position)
        self.figure = figures['Knight']

    def valid_moves(self, board_instance):
        moves = []
        row, col = self.position

        dc = [[1, 2], [1, -2], [-1, 2], [-1, -2], [2, 1], [2, -1], [-2, 1], [-2, -1]]

        for i in range(8):
            new_row, new_col = row + dc[i][0], col + dc[i][1]
            if 0 <= new_row < 8 and 0 <= new_col < 8 and not board_instance.check_for_friendly((new_row, new_col), self.color):
                moves.append((new_row, new_col))

        return moves

In [729]:
class Bishop(Piece):
    def __init__(self, color, position):
        super().__init__(color, position)
        self.figure = figures['Bishop']

    def valid_moves(self, board_instance):
        moves = []
        row, col = self.position

        directions = [[1, 1], [1, -1], [-1, 1], [-1, -1]]

        for dr, dc in directions:
            r, c = row + dr, col + dc
            while 0 <= r < 8 and 0 <= c < 8:
                if board_instance.check_for_friendly((r, c), self.color):
                    break
                moves.append((r, c))
                if board_instance.check_for_enemy((r, c), self.color):
                    break
                r += dr
                c += dc

        return moves

In [730]:
class Queen(Piece):
    def __init__(self, color, position):
        super().__init__(color, position)
        self.figure = figures['Queen']

    def valid_moves(self, board_instance):
        moves = []
        row, col = self.position

        directions = [[1, 0], [-1, 0], [0, 1], [0, -1], [1, 1], [1, -1], [-1, 1], [-1, -1]]

        for dr, dc in directions:
            r, c = row + dr, col + dc
            while 0 <= r < 8 and 0 <= c < 8:
                if board_instance.check_for_friendly((r, c), self.color):
                    break
                moves.append((r, c))
                if board_instance.check_for_enemy((r, c), self.color):
                    break
                r += dr
                c += dc

        return moves

In [731]:
class King(Piece):
    def __init__(self, color, position):
        super().__init__(color, position)
        self.figure = figures['King']
        self.destructable = False

    def valid_moves(self, board_instance):
        moves = []
        row, col = self.position

        dc = [[1, 0], [-1, 0], [0, 1], [0, -1], [1, 1], [1, -1], [-1, 1], [-1, -1]]

        for i in range(8):
            new_row, new_col = row + dc[i][0], col + dc[i][1]
            if 0 <= new_row < 8 and 0 <= new_col < 8 and not board_instance.check_for_friendly((new_row, new_col), self.color):
                moves.append((new_row, new_col))

        return moves

In [732]:
class Board:
    def __init__(self):
        self.board = self.create_board()

    def create_board(self):
        return [[None for _ in range(8)] for _ in range(8)]

    def populate_board(self):
        # Place Pawns
        for col in range(8):
            self.board[1][col] = Pawn(colors['white'], (1, col))
            self.board[6][col] = Pawn(colors['black'], (6, col))

        # Place Rooks
        self.board[0][0] = Rook(colors['white'], (0, 0))
        self.board[0][7] = Rook(colors['white'], (0, 7))
        self.board[7][0] = Rook(colors['black'], (7, 0))
        self.board[7][7] = Rook(colors['black'], (7, 7))

        # Place Knights
        self.board[0][1] = Knight(colors['white'], (0, 1))
        self.board[0][6] = Knight(colors['white'], (0, 6))
        self.board[7][1] = Knight(colors['black'], (7, 1))
        self.board[7][6] = Knight(colors['black'], (7, 6))

        # Place Bishops
        self.board[0][2] = Bishop(colors['white'], (0, 2))
        self.board[0][5] = Bishop(colors['white'], (0, 5))
        self.board[7][2] = Bishop(colors['black'], (7, 2))
        self.board[7][5] = Bishop(colors['black'], (7, 5))

        # Place Queens
        self.board[0][3] = Queen(colors['white'], (0, 3))
        self.board[7][3] = Queen(colors['black'], (7, 3))

        # Place Kings
        self.board[0][4] = King(colors['white'], (0, 4))
        self.board[7][4] = King(colors['black'], (7, 4))

    def show_board(self):
        print("  " + " ".join(str(x) for x in range(8)))
        for y, row in enumerate(self.board):
            print(f"{y} " + " ".join([reverse_figures[piece.figure] if piece else '.' for piece in row]))

    def get_valid_moves(self, position):
        piece = self.get_piece(position)
        if piece:
            return piece.valid_moves(self)
        return []

    def get_piece(self, position):
        row, col = position
        piece = self.board[row][col]
        return piece

    def check_for_enemy(self, position, color):
        piece = self.get_piece(position)
        if piece:
            return piece.color != color
        return False

    def check_for_friendly(self, position, color):
        piece = self.get_piece(position)
        if piece:
            return piece.color == color
        return False

In [733]:
board = Board()
board.populate_board()

board.show_board()

  0 1 2 3 4 5 6 7
0 R K B Q K B K R
1 P P P P P P P P
2 . . . . . . . .
3 . . . . . . . .
4 . . . . . . . .
5 . . . . . . . .
6 P P P P P P P P
7 R K B Q K B K R


In [734]:
valid_moves = board.get_valid_moves([0, 6])
print(valid_moves)

[(2, 7), (2, 5)]


In [735]:
board.board[1] = [None for _ in range(8)]
board.show_board()

  0 1 2 3 4 5 6 7
0 R K B Q K B K R
1 . . . . . . . .
2 . . . . . . . .
3 . . . . . . . .
4 . . . . . . . .
5 . . . . . . . .
6 P P P P P P P P
7 R K B Q K B K R


In [736]:
valid_moves = board.get_valid_moves([0, 0])
print(valid_moves)

[(1, 0), (2, 0), (3, 0), (4, 0), (5, 0), (6, 0)]


In [737]:
import pygame
import sys

In [738]:
import pygame
import sys

# Initialize Pygame
pygame.init()

# Constants
WIDTH, HEIGHT = 800, 800
ROWS, COLS = 8, 8
SQUARE_SIZE = WIDTH // COLS

WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
LIGHT_BROWN = (240, 217, 181)
DARK_BROWN = (181, 136, 99)
BLUE = (0, 0, 255)

WIN = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption('Chess')

def load_images():
    pieces = {
        'wp': 'white pawn',
        'bp': 'black pawn',
        'wr': 'white rook',
        'br': 'black rook',
        'wn': 'white knight',
        'bn': 'black knight',
        'wb': 'white bishop',
        'bb': 'black bishop',
        'wq': 'white queen',
        'bq': 'black queen',
        'wk': 'white king',
        'bk': 'black king'
    }
    images = {}
    for piece, name in pieces.items():
        file_name = name.replace(' ', '_') + '.png'
        try:
            images[piece] = pygame.transform.scale(pygame.image.load(f'images/{file_name}'), (SQUARE_SIZE, SQUARE_SIZE))
        except FileNotFoundError:
            print(f"File not found: images/{file_name}")
    return images

IMAGES = load_images()

def draw_board(win):
    win.fill(WHITE)
    for row in range(ROWS):
        for col in range(COLS):
            color = LIGHT_BROWN if (row + col) % 2 == 0 else DARK_BROWN
            pygame.draw.rect(win, color, (col * SQUARE_SIZE, row * SQUARE_SIZE, SQUARE_SIZE, SQUARE_SIZE))

def draw_pieces(win, board):
    for row in range(ROWS):
        for col in range(COLS):
            piece = board[row][col]
            if piece:
                piece_key = f"{'w' if piece.color == 0 else 'b'}{'n' if piece.__class__.__name__ == 'Knight' else piece.__class__.__name__[0].lower()}"
                win.blit(IMAGES[piece_key], (col * SQUARE_SIZE, row * SQUARE_SIZE))

def highlight_moves(win, moves):
    for move in moves:
        row, col = move
        pygame.draw.circle(win, BLUE, (col * SQUARE_SIZE + SQUARE_SIZE // 2, row * SQUARE_SIZE + SQUARE_SIZE // 2), SQUARE_SIZE // 4)

def main():
    board_instance = board

    selected_piece = None
    valid_moves = []

    clock = pygame.time.Clock()
    run = True
    while run:
        clock.tick(60)
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False
            elif event.type == pygame.MOUSEBUTTONDOWN:
                pos = pygame.mouse.get_pos()
                col, row = pos[0] // SQUARE_SIZE, pos[1] // SQUARE_SIZE
                selected_piece = board_instance.get_piece((row, col))
                if selected_piece:
                    valid_moves = board_instance.get_valid_moves((row, col))
                else:
                    valid_moves = []

        draw_board(WIN)
        draw_pieces(WIN, board_instance.board)
        highlight_moves(WIN, valid_moves)
        pygame.display.flip()

    pygame.quit()
    sys.exit()

if __name__ == "__main__":
    main()

SystemExit: 